In [34]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# 괄호로 감싸진 숫자를 음수로 변환하는 함수
def convert_to_numeric(value):
    """괄호로 감싸진 숫자를 음수로 변환하는 함수"""
    if pd.isna(value):
        return value
    if isinstance(value, str):
        value_clean = value.strip()
        # 괄호로 감싸진 경우: (3000) -> -3000
        if value_clean.startswith('(') and value_clean.endswith(')'):
            value_clean = '-' + value_clean[1:-1]
        # 쉼표 제거
        value_clean = value_clean.replace(',', '').strip()
        return pd.to_numeric(value_clean, errors='coerce')
    return pd.to_numeric(value, errors='coerce')

# 1. CSV 파일 읽기
# 프로젝트 루트 경로 설정 (절대 경로 사용)
project_root = '/Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction'

# 상대 경로로도 시도 (노트북이 imputation/field_per/에 있을 때)
if not os.path.exists(project_root):
    # 현재 작업 디렉토리에서 2단계 위로
    current_dir = os.getcwd()
    if 'imputation' in current_dir or 'field_per' in current_dir:
        project_root = os.path.abspath(os.path.join(current_dir, '../..'))
    else:
        # 다른 위치에서 실행되는 경우
        project_root = os.path.abspath(os.path.join(current_dir, '..'))

csv_path = os.path.join(project_root, 'data', '재무정보_final_v3.csv')

# 경로 확인 및 출력
print(f"프로젝트 루트: {project_root}")
print(f"CSV 파일 경로: {csv_path}")
print(f"파일 존재 여부: {os.path.exists(csv_path)}")

if not os.path.exists(csv_path):
    csv_path = '/Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction/data/재무정보_final_v3.csv'
    print(f"대체 경로 사용: {csv_path}")

df = pd.read_csv(csv_path, encoding='utf-8')
data = df.copy()

# 컬럼명의 앞뒤 공백 제거
df.columns = df.columns.str.strip()

print("=== 원본 데이터 정보 ===")
print(f"데이터 shape: {df.shape}")
print(f"\n컬럼 목록:")
print(df.columns.tolist())
print(f"\nfield별 분포:")
print(df['field'].value_counts())
print(f"\n연도별 분포:")
print(df['연도'].value_counts().sort_index())

# 2. 숫자형 컬럼 선택 (결측치를 채울 컬럼들)
# 사용자 요구사항: 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, 연구개발비, CAPEX, 유형자산_당기, 유형자산_전기, 유형자산_증감
numeric_cols = ['매출액', '영업이익', '당기순이익', '자산총계', '부채총계', '자본총계', 
                '연구개발비', 'CAPEX', '유형자산_당기', '유형자산_전기', '유형자산_증감']

# 존재하는 컬럼만 선택
numeric_cols = [col for col in numeric_cols if col in df.columns]

# 숫자형 컬럼을 숫자형으로 변환 (문자열이나 object 타입인 경우)
# 괄호 처리: (3000) -> -3000, 쉼표와 공백 제거 후 변환
for col in numeric_cols:
    if df[col].dtype == 'object':
        # 문자열인 경우 괄호 처리, 쉼표와 공백 제거
        df[col] = df[col].apply(convert_to_numeric)
    else:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"\n=== 결측치 채울 컬럼 ({len(numeric_cols)}개) ===")
print(numeric_cols)

print(f"\n=== 결측치 현황 ===")
missing_info = df[numeric_cols].isnull().sum()
print(missing_info[missing_info > 0])

프로젝트 루트: /Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction
CSV 파일 경로: /Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction/data/재무정보_final_v3.csv
파일 존재 여부: True
=== 원본 데이터 정보 ===
데이터 shape: (4194, 18)

컬럼 목록:
['기업명', '사업자등록번호', 'crno', 'dart_corp_code', 'sectors', 'field', '연도', '매출액', '영업이익', '당기순이익', '자산총계', '부채총계', '자본총계', '연구개발비', 'CAPEX', '유형자산_당기', '유형자산_전기', '유형자산_증감']

field별 분포:
field
부품    1971
장비    1629
소재     594
Name: count, dtype: int64

연도별 분포:
연도
2016    466
2017    466
2018    466
2019    466
2020    466
2021    466
2022    466
2023    466
2024    466
Name: count, dtype: int64

=== 결측치 채울 컬럼 (11개) ===
['매출액', '영업이익', '당기순이익', '자산총계', '부채총계', '자본총계', '연구개발비', 'CAPEX', '유형자산_당기', '유형자산_전기', '유형자산_증감']

=== 결측치 현황 ===
매출액        1360
영업이익       1361
당기순이익      1350
자산총계       1344
부채총계       1346
자본총계       1348
연구개발비      2385
CAPEX      2167
유형자산_당기    2986
유형자산_전기    3014
유형자산_증감    2990
dtype: int64


In [35]:
# 3. field별, 연도별로 상하위 10% 기업들의 평균으로 결측치 채우기
print(f"\n=== field별 상하위 10% 평균으로 결측치 채우기 시작 ===")

df_final = df.copy()

# df_final의 숫자형 컬럼도 숫자형으로 변환 (괄호 처리 포함)
for col in numeric_cols:
    if df_final[col].dtype == 'object':
        df_final[col] = df_final[col].apply(convert_to_numeric)
    else:
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

# 각 기업별로 처리
companies = df['기업명'].unique()
print(f"총 {len(companies)}개 기업 처리 중...")

for company in tqdm(companies, desc="기업 처리"):
    company_data = df[df['기업명'] == company].copy()
    company_field = company_data['field'].iloc[0] if company_data['field'].notna().any() else None
    
    if company_field is None:
        continue
    
    # 해당 기업의 각 연도별로 처리
    for target_year in range(2016, 2025):
        company_year_data = company_data[company_data['연도'] == target_year]
        
        if len(company_year_data) == 0:
            continue
        
        company_idx = company_year_data.index[0]
        
        # 각 숫자형 컬럼에 대해 처리
        for col in numeric_cols:
            # 이미 값이 있으면 스킵
            if pd.notna(df_final.loc[company_idx, col]):
                continue
            
            # 참조값 찾기 로직: 2016~2024년도 중 값이 있는 모든 연도들의 평균
            reference_value = None
            ref_values = []
            
            # 2016~2024년도 중 값이 있는 모든 연도 찾기
            for year in range(2016, 2025):
                year_data = company_data[company_data['연도'] == year]
                if len(year_data) > 0:
                    year_idx = year_data.index[0]
                    # 원본 데이터에서 찾기
                    ref_val = df.loc[year_idx, col]
                    # 괄호 처리 및 숫자형 변환
                    ref_val = convert_to_numeric(ref_val)
                    if pd.notna(ref_val) and np.isfinite(ref_val):
                        ref_values.append(ref_val)
            
            # 값이 있는 연도들의 평균을 참조값으로 사용
            if len(ref_values) > 0:
                reference_value = np.mean(ref_values)
            
            # 동일한 field의 다른 기업들 중에서
            # 해당 연도(target_year)에 값이 있는 기업들 찾기 (df_final에서 찾아야 이미 채워진 값도 반영)
            field_year_data = df_final[(df_final['field'] == company_field) & 
                                      (df_final['연도'] == target_year) & 
                                      (df_final[col].notna())].copy()
            
            # 자기 자신은 제외
            field_year_data = field_year_data[field_year_data['기업명'] != company]
            
            if len(field_year_data) == 0:
                continue
            
            # 참조값과의 차이 계산
            field_year_data[col] = pd.to_numeric(field_year_data[col], errors='coerce')
            field_year_data = field_year_data[field_year_data[col].notna()].copy()
            
            if len(field_year_data) == 0:
                continue
            
            # 참조값이 있는 경우: 참조값과 가까운 상하위 10% 선택
            if reference_value is not None:
                reference_value = pd.to_numeric(reference_value, errors='coerce')
                if pd.notna(reference_value):
                    field_year_data['diff'] = abs(field_year_data[col] - reference_value)
                    field_year_data = field_year_data.sort_values('diff')
                    
                    # 상하위 10% 선택 (최소 1개는 선택)
                    n_select = max(1, int(len(field_year_data) * 0.1))
                    if n_select == 0:
                        n_select = 1
                    
                    # 상하위 10% 기업들의 평균 계산
                    selected_data = field_year_data.head(n_select)
                    avg_value = selected_data[col].mean()
                else:
                    # 참조값이 유효하지 않으면 전체 평균 사용
                    avg_value = field_year_data[col].mean()
            else:
                # 참조값이 없으면 (모든 연도가 NaN인 경우) 해당 field의 해당 연도 전체 평균 사용
                avg_value = field_year_data[col].mean()
            
            # 평균값이 유효한 숫자인지 확인
            if pd.notna(avg_value) and np.isfinite(avg_value):
                # 평균값을 정수로 변환 (소수점 제거)
                avg_value_int = int(round(avg_value))
                # 결측치 채우기
                df_final.loc[company_idx, col] = avg_value_int

print(f"\n=== Imputation 완료 ===")
print(f"최종 데이터 shape: {df_final.shape}")


=== field별 상하위 10% 평균으로 결측치 채우기 시작 ===
총 466개 기업 처리 중...


기업 처리: 100%|██████████| 466/466 [00:24<00:00, 18.83it/s]


=== Imputation 완료 ===
최종 데이터 shape: (4194, 18)


In [36]:
# 4. 결과 확인
print(f"\n=== Imputation 후 결측치 현황 ===")
missing_after = df_final[numeric_cols].isnull().sum()
print(missing_after[missing_after > 0])
if missing_after.sum() == 0:
    print("모든 결측치가 채워졌습니다!")
else:
    print(f"\n남은 결측치: {missing_after.sum()}개")

# 샘플 데이터 확인
print(f"\n=== Imputation 결과 샘플 (첫 번째 기업) ===")
first_company = df_final['기업명'].iloc[0]
sample_data = df_final[df_final['기업명'] == first_company][['기업명', '연도', 'field'] + numeric_cols]
print(sample_data.head(10))


=== Imputation 후 결측치 현황 ===
Series([], dtype: int64)
모든 결측치가 채워졌습니다!

=== Imputation 결과 샘플 (첫 번째 기업) ===
         기업명    연도 field           매출액         영업이익        당기순이익  \
0  (주)나노젯코리아  2016    장비  3.850259e+09  337597817.0  284507692.0   
1  (주)나노젯코리아  2017    장비  4.569928e+09  292823910.0  225081921.0   
2  (주)나노젯코리아  2018    장비  4.419109e+09  370715323.0  281307917.0   
3  (주)나노젯코리아  2019    장비  1.675000e+09   66000000.0   59000000.0   
4  (주)나노젯코리아  2020    장비  5.141000e+09  345000000.0  317000000.0   
5  (주)나노젯코리아  2021    장비  7.011000e+09  558000000.0  503000000.0   
6  (주)나노젯코리아  2022    장비  4.305901e+09  326453308.0  295073192.0   
7  (주)나노젯코리아  2023    장비  4.537873e+09  356025194.0  258318187.0   
8  (주)나노젯코리아  2024    장비  4.752615e+09  337605313.0  282693027.0   

           자산총계          부채총계         자본총계         연구개발비         CAPEX  \
0  1.853898e+09  1.305383e+09  514350617.0  3.124368e+09  2.426534e+09   
1  1.949775e+09  1.236623e+09  444377920.0  3.766336e+09  3.12201

In [37]:
# 5. 결과 저장
# project_root가 제대로 설정되었는지 확인
try:
    if 'project_root' not in globals() or not os.path.exists(project_root):
        project_root = '/Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction'
except:
    project_root = '/Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction'

output_path = os.path.join(project_root, 'data', '재무정보_final_imputed.csv')
print(f"\n=== 결과 저장 ===")
print(f"프로젝트 루트: {project_root}")
print(f"저장 경로: {output_path}")
print(f"디렉토리 존재 여부: {os.path.exists(os.path.dirname(output_path))}")

# 디렉토리가 없으면 생성
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# cp949로 저장하기 전에 문제가 되는 문자 제거
df_final_clean = df_final.copy()
for col in df_final_clean.columns:
    if df_final_clean[col].dtype == 'object':
        # 문자열 컬럼에서 non-breaking space(\xa0)를 일반 공백으로 변환
        df_final_clean[col] = df_final_clean[col].astype(str).str.replace('\xa0', ' ', regex=False)

# cp949로 저장 (errors='replace'로 인코딩 불가능한 문자 대체)
try:
    df_final_clean.to_csv(output_path, index=False, encoding='cp949', errors='replace')
    
    # 저장 확인
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path)
        print(f"\n=== 결과 저장 완료 ===")
        print(f"저장 경로: {output_path}")
        print(f"저장된 데이터 shape: {df_final.shape}")
        print(f"파일 크기: {file_size:,} bytes")
        print(f"저장 성공!")
    else:
        print(f"\n=== 저장 실패 ===")
        print(f"파일이 생성되지 않았습니다: {output_path}")
except Exception as e:
    print(f"\n=== 저장 중 오류 발생 ===")
    print(f"오류 내용: {e}")
    print(f"저장 경로: {output_path}")

# 데이터 요약 통계
print(f"\n=== Imputation 전후 비교 ===")
print("\n[Imputation 전]")
print(f"  총 결측치: {df[numeric_cols].isnull().sum().sum()}개")
print(f"  결측치 비율: {df[numeric_cols].isnull().sum().sum() / (len(df) * len(numeric_cols)) * 100:.2f}%")

print("\n[Imputation 후]")
print(f"  총 결측치: {df_final[numeric_cols].isnull().sum().sum()}개")
print(f"  결측치 비율: {df_final[numeric_cols].isnull().sum().sum() / (len(df_final) * len(numeric_cols)) * 100:.2f}%")


=== 결과 저장 ===
프로젝트 루트: /Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction
저장 경로: /Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction/data/재무정보_final_imputed.csv
디렉토리 존재 여부: True

=== 결과 저장 완료 ===
저장 경로: /Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction/data/재무정보_final_imputed.csv
저장된 데이터 shape: (4194, 18)
파일 크기: 894,697 bytes
저장 성공!

=== Imputation 전후 비교 ===

[Imputation 전]
  총 결측치: 21651개
  결측치 비율: 46.93%

[Imputation 후]
  총 결측치: 0개
  결측치 비율: 0.00%
